In [1]:
from LSTM_Model.core import data_processor, model as core_model, utils
import json
import os
import math
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# add header for all the data

market_data_path = "LSTM_Model/data/market_data"
for tournament in list(os.listdir(market_data_path)):
    if tournament[0]==".":
        continue
    tournament_path = os.path.join(market_data_path, tournament)
    for team in list(os.listdir(tournament_path)):
            if team[0]=="." or team=="team_info.csv":
                continue
            team_path = os.path.join(tournament_path, team)
            rows = [["Date","Price","Volume"]]
            with open(team_path,"r",newline="") as csvfile:
                csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
                for row in csvreader:
                    rows.append(row)
            os.remove(team_path)
            with open(team_path,"w",newline="") as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=',', quotechar='|')
                for row in rows:
                    csvwriter.writerow(row)

In [2]:
with open("sticker_model_config.json", 'r') as myfile:
    config_text=myfile.read().replace('\n', '')
configs = json.loads(config_text)
cwd = os.getcwd()
print(os.path.join(cwd+"/LSTM_Model/data/", configs['data']['filename']))
print(configs['data']['train_test_split'])
print(configs['data']['columns'])

#model = core_model.Model()
#model.build_model(configs)

/home/yuhongc_berkeley_edu/csgo_stickers/LSTM_Model/data/market_data/Cologne 2014/Cloud9.csv
0.9
['Price']


In [3]:
market_data_path = "LSTM_Model/data/market_data"
for tournament in list(os.listdir(market_data_path)):
    if tournament[0]=="." or tournament=="Boston 2018":
        continue
    tournament_path = os.path.join(market_data_path, tournament)
    for team in list(os.listdir(tournament_path)):
            if team[0]=="." or team=="team_info.csv":
                continue
            team_path = os.path.join(tournament_path, team)
            
            # for each team, create a different DataLoader
            data = data_processor.DataLoader(
                team_path,
                configs['data']['train_test_split'],
                configs['data']['columns']
            )
            
            # get training data
            x, y = data.get_train_data(
                seq_len = configs['data']['sequence_length'],
                normalise = configs['data']['normalise']
            )
            
            # train model with data
            model.train(
                x,
                y,
                epochs = configs['training']['epochs'],
                batch_size = configs['training']['batch_size'],
                save_dir="Saved_Models"
            )
            

[Model] Training Started
[Model] 2 epochs, 32 batch size
Epoch 1/2
678/678 [==============================] - 4s 7ms/step - loss: 0.0172
Epoch 2/2
678/678 [==============================] - 2s 4ms/step - loss: 0.0081
[Model] Training Completed. Model saved as Saved_Models/04112018-041157-e2.h5
Time taken: 0:00:09.284488
[Model] Training Started
[Model] 2 epochs, 32 batch size
Epoch 1/2
619/619 [==============================] - 2s 4ms/step - loss: 0.0087
Epoch 2/2
619/619 [==============================] - 2s 4ms/step - loss: 0.0083
[Model] Training Completed. Model saved as Saved_Models/04112018-041206-e2.h5
Time taken: 0:00:04.465649
[Model] Training Started
[Model] 2 epochs, 32 batch size
Epoch 1/2
643/643 [==============================] - 2s 4ms/step - loss: 0.0122
Epoch 2/2
643/643 [==============================] - 2s 4ms/step - loss: 0.0104
[Model] Training Completed. Model saved as Saved_Models/04112018-041211-e2.h5
Time taken: 0:00:04.652770
[Model] Training Started
[Model] 2

In [4]:
def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()

In [6]:
boston_path = os.path.join(market_data_path, "Boston 2018")
team_path = os.path.join(boston_path, "Astralis.csv")
data = data_processor.DataLoader(
        team_path,
        0,
        configs['data']['columns']
    )
x_test, y_test = data.get_test_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )

In [3]:
model = core_model.Model()
model.load_model("Saved_Models/04112018-042955-e2.h5")

[Model] Loading model from file Saved_Models/04112018-042955-e2.h5


In [9]:
model_predictions=[]
market_data_path = "LSTM_Model/data/market_data"
boston_path = os.path.join(market_data_path, "Boston 2018")
for team in list(os.listdir(boston_path)):
    if team[0]=="." or team=="team_info.csv":
        continue
    team_path = os.path.join(boston_path, team)
    
    # create DataLoader for each team
    data = data_processor.DataLoader(
            team_path,
            0,
            configs['data']['columns']
        )
    x_test, y_test = data.get_test_data(
        seq_len=configs['data']['sequence_length'],
        normalise=configs['data']['normalise']
    )
    predictions_fullseq = model.predict_sequence_full(x_test[:365], configs['data']['sequence_length']) # prediction of the value in 365 days
    final_prediction = predictions_fullseq[-1]
    model_predictions.append([team[:-4], final_prediction])
model_predictions
    

[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...
[Model] Predicting Sequences Full...


[['Renegades', -0.4436195],
 ['Team EnVyUs', -0.44364458],
 ['Vega Squadron', -0.44338667],
 ['mousesports', -0.44389594],
 ['G2 Esports', -0.4435767],
 ['Space Soldiers', -0.44186252],
 ['Flash Gaming', -0.44374144],
 ['SK Gaming', -0.44292963],
 ['Misfits Gaming', -0.44076353],
 ['Natus Vincere', -0.44343883],
 ['Team Liquid', -0.44350803],
 ['Virtus.Pro', -0.44256705],
 ['Avangar', -0.442537],
 ['Astralis', -0.44290817],
 ['Flipsid3 Tactics', -0.44313174],
 ['Gambit', -0.4434355],
 ['Quantum Bellator Fire', -0.44359624],
 ['BIG', -0.44323993],
 ['Cloud9', -0.44137204],
 ['North', -0.44281685],
 ['Sprout Esports', -0.4427268],
 ['FaZe Clan', -0.44290334],
 ['Fnatic', -0.4428075]]

In [11]:
with open("lstm_predictions.csv","w",newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csv
    for prediction in model_predictions:
        csvwriter.writerow(prediction)